In [2]:
import yahoofinancials as yf    
from quantifin.util.markets import Market
from quantifin.equity.valuation import *
from quantifin.equity import Stock
import json

In [3]:
jpm = Stock("JPM")
nyse = Market(jpm.get_stock_exchange())
price_list = nyse.get_historical_price_data("2010-01-01", "2021-01-01", 'monthly')

In [4]:
market_returns = nyse.get_annualised_return(3)
market_returns


0.0613

In [5]:
#Gordon Growth Model
current_dividend = jpm.full_year_dividend()
growth_rate = jpm.growth_rate()
req_rate = CAPM(0.05, market_returns , jpm.beta)
print(f"Current dividend: {current_dividend} growth rate: {growth_rate} req return {req_rate}")
gordon_growth_valuation(current_dividend, req_rate, growth_rate * 0.5)

Current dividend: 3.6 growth rate: 0.074 req return 0.06357713079999999


140.467

In [6]:
#Forward PE
payout_hist = jpm.get_dividend_payout_ratio_history()
payout = jpm.average_dividend_payout_ratio(payout_hist)
growth_rate = jpm.growth_rate() * 0.5
eps = jpm.get_key_statistics_data()[jpm.stock_code]['trailingEps']
print(f"payout {payout} growth {growth_rate} req_rate {req_rate} EPS {eps}")
forward_pe(payout, growth_rate, req_rate, eps)

AttributeError: 'Stock' object has no attribute 'YfApi'